In [6]:
using TensorDecomposition, StatsBase, LinearAlgebra, Combinatorics, InvertedIndices, Einsum, IterTools, HomotopyContinuation

In [7]:
function from_multiindex(x, n)
    d = length(x)
    c = 0
    for i=1:d-1
        c += (x[i]-1)*n^(d-i)
    end
    return c + x[d]
end;

function kronMat2(A, d)
   return foldl(kron, repeat([A], d)) 
end

function kronMatDrop(A, d)
    n = size(A)[1]-1
    inds = collect(with_replacement_combinations(1:n+1, d))
    inds = map(x->from_multiindex(x, n+1), inds)
    return kronMat2(A, d)[inds, inds]
end;

function khatri_rao(A, d)
    if d == 1
        B = A
    else
        n, r = size(A)
        B = zeros(eltype(A), (n^d, r))
        for i=1:r 
            B[:, i] = kron(ntuple(x->A[:, i], d)...)
        end;
    end;
    return B
end

function krDrop(A, d)
    n = size(A)[1]-1
    inds = collect(with_replacement_combinations(1:n+1, d))
    inds = map(x->from_multiindex(x, n+1), inds)
    return khatri_rao(A, d)[inds, :]
end;

function changeBasis(T, M)
    Tsize = size(T)
    n1 = Tsize[1]
    d = length(Tsize)
    
    n2 = size(M)[1]
    
    That = reshape(T, n1^d)
    return reshape(foldl(kron, repeat([M], d))*That, repeat([n2], d)...)
end;

function multMon(x, j)
    c = copy(reverse(x))
    for i=1:length(c)
        if c[i] == 0
            c[i] = j
            break
        end
    end;
    return sort(reverse(c))
end;

## Stanley's example

In [8]:
n = 12
d = 4;

In [9]:
T = zeros(repeat([n+1], d)...);
first_parts = collect(with_replacement_combinations(1:3, 3))
vars = Set()
for (i, first_part) in enumerate(first_parts)
    push!(vars, append!(first_part, i+3))
end

In [10]:
for ind in with_replacement_combinations(1:n+1, 4)
    if ind in vars
        perms = Combinatorics.permutations(ind)
        for perm in perms
            T[perm...] = 1 
        end
    end
end

In [11]:
M = randn(n+1, n+1);

In [12]:
k = 1

rowInds = collect(with_replacement_combinations(1:n+1, k))
rowInds = map(x->from_multiindex(x, n+1), rowInds)

colInds = collect(with_replacement_combinations(1:n+1, d-k))
colInds = map(x->from_multiindex(x, n+1), colInds)

catMatM = (kronMat2(M, k)*reshape(T, ((n+1)^k, (n+1)^(d-k)))*permutedims(kronMat2(M, d-k)))[rowInds, colInds];

In [13]:
k = 2

rowInds = collect(with_replacement_combinations(1:n+1, k))
rowInds = map(x->from_multiindex(x, n+1), rowInds)

colInds = collect(with_replacement_combinations(1:n+1, d-k))
colInds = map(x->from_multiindex(x, n+1), colInds)

catMatM = (kronMat2(M, k)*reshape(T, ((n+1)^k, (n+1)^(d-k)))*permutedims(kronMat2(M, d-k)))[rowInds, colInds];

In [14]:
rank(catMatM[1:13, 1:13])

6

In [15]:
catMat(T, 2)

91×91 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [16]:
catMat(T, 1)

13×455 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

## Perazzo's cubic

In [12]:
n = 4
d = 4;

In [13]:
T = zeros(repeat([n+1], d)...);

inds = Set([[1, 4, 4, 4], [2, 4, 5, 5], [3, 4, 4, 5]])
for ind in with_replacement_combinations(1:n+1, d)
    if ind in inds
        perms = Combinatorics.permutations(ind)
        for perm in perms
            T[perm...] = 1 
        end
    end
end

In [14]:
M = randn(n+1, n+1);
TM = changeBasis(T, M);

In [15]:
Q, R = qr(catMat(T, 2));

In [16]:
catMat(T, 2)[[4, 8, 11, 13, 14, 15], [4, 8, 11, 13, 14, 15]]

6×6 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0

In [17]:
QM, RM = qr(catMat(TM, 2));
rank(catMat(TM, 2)[1:5, :])

5

## Monomials

In [17]:
n = 3
d = 4
r = 8;

In [50]:
P = zeros(n+1, r)
P[1, :] = ones(r)
sigs = product(repeat([[-1, 1]], n)...)
for (i, sig) in enumerate(sigs)
    P[2:end, i] .= sig
end

L = prod.(eachcol(P)) ;

In [80]:
T = makeRankedTensor(L, P, d+2) / 8;
Tcat = catMat(T, 3);

In [81]:
A = (permutedims(Complex.(L) .^ (1/6)) .* P ) / r^(1/6)

4×8 Matrix{ComplexF64}:
  0.612372+0.353553im   0.707107+0.0im   0.707107+0.0im  …  0.707107+0.0im
 -0.612372-0.353553im   0.707107+0.0im  -0.707107-0.0im     0.707107+0.0im
 -0.612372-0.353553im  -0.707107-0.0im   0.707107+0.0im     0.707107+0.0im
 -0.612372-0.353553im  -0.707107-0.0im  -0.707107-0.0im     0.707107+0.0im

In [98]:
A2 = khatri_rao(A, 2)

16×8 Matrix{ComplexF64}:
  0.25+0.433013im   0.5+0.0im   0.5+0.0im  …   0.25+0.433013im  0.5+0.0im
 -0.25-0.433013im   0.5+0.0im  -0.5-0.0im     -0.25-0.433013im  0.5+0.0im
 -0.25-0.433013im  -0.5-0.0im   0.5+0.0im      0.25+0.433013im  0.5+0.0im
 -0.25-0.433013im  -0.5-0.0im  -0.5-0.0im      0.25+0.433013im  0.5+0.0im
 -0.25-0.433013im   0.5+0.0im  -0.5-0.0im     -0.25-0.433013im  0.5+0.0im
  0.25+0.433013im   0.5+0.0im   0.5+0.0im  …   0.25+0.433013im  0.5+0.0im
  0.25+0.433013im  -0.5-0.0im  -0.5-0.0im     -0.25-0.433013im  0.5+0.0im
  0.25+0.433013im  -0.5-0.0im   0.5+0.0im     -0.25-0.433013im  0.5+0.0im
 -0.25-0.433013im  -0.5-0.0im   0.5+0.0im      0.25+0.433013im  0.5+0.0im
  0.25+0.433013im  -0.5-0.0im  -0.5-0.0im     -0.25-0.433013im  0.5+0.0im
  0.25+0.433013im   0.5+0.0im   0.5+0.0im  …   0.25+0.433013im  0.5+0.0im
  0.25+0.433013im   0.5+0.0im  -0.5-0.0im      0.25+0.433013im  0.5+0.0im
 -0.25-0.433013im  -0.5-0.0im  -0.5-0.0im      0.25+0.433013im  0.5+0.0im
  0.25+0.4330

In [90]:
M = randn(n+1, n+1)

4×4 Matrix{Float64}:
 -1.33609    0.929436   0.733894   -0.685808
 -1.67814   -0.245635  -0.787027    0.146802
 -0.136182  -0.303429  -2.11711     1.50568
 -0.991588  -1.31221    0.0251195  -0.918185

In [95]:
I = diagm(ones(n+1));
IM = kron(I, M)
MI = kron(M, I);

In [105]:
rank(A2[1:10, :])

6

In [129]:
rank((kron(I, M)*A2)[[1, 2, 3, 4, 5, 6, 10], :])

7

In [179]:
V1 = svd((IM*A2)[[1, 2, 3, 4, 5, 6, 9], :], full=true).V

8×8 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
  -0.171709+0.297409im     -0.0183194+0.0317301im    …    0.2374+0.261994im
 -0.0406259+9.79202e-18im    0.302092-5.25629e-17im     0.108194-0.336592im
   0.227893-5.31782e-17im  -0.0487302-4.09423e-17im     0.108194-0.336592im
   0.151927-0.263145im       0.295683-0.512138im          0.2374+0.261994im
  -0.759903+1.03714e-16im   0.0918084-6.24357e-17im     0.108194-0.336592im
   -0.19176+0.332139im       0.200831-0.347849im     …    0.2374+0.261994im
 -0.0300805+0.052101im     0.00481608-0.00834169im        0.2374+0.261994im
  0.0893895-1.5329e-17im     0.620851+1.58669e-17im     0.108194-0.336592im

In [196]:
rank((IM*A2)[[1, 2, 3, 5, 6, 7], :])

6

In [229]:
MI[[1, 2, 3, 4, 5, 6, 7], 1:7]

7×7 Matrix{Float64}:
 -1.33609  -0.0      -0.0      -0.0       0.929436   0.0        0.0
 -0.0      -1.33609  -0.0      -0.0       0.0        0.929436   0.0
 -0.0      -0.0      -1.33609  -0.0       0.0        0.0        0.929436
 -0.0      -0.0      -0.0      -1.33609   0.0        0.0        0.0
 -1.67814  -0.0      -0.0      -0.0      -0.245635  -0.0       -0.0
 -0.0      -1.67814  -0.0      -0.0      -0.0       -0.245635  -0.0
 -0.0      -0.0      -1.67814  -0.0      -0.0       -0.0       -0.245635

In [234]:
(IM*A2)[1:7, :]

7×8 Matrix{ComplexF64}:
 -0.578403-1.00182im    -0.22737+0.0im  -0.422912+0.0im  …  -0.179284+0.0im
  -0.19807-0.343067im  -0.641775+0.0im   -1.18317+0.0im        -1.282+0.0im
  0.194669+0.337176im  0.0859088+0.0im   -1.72777+0.0im      -0.52552+0.0im
   0.30342+0.52554im   -0.705364+0.0im    0.63196+0.0im      -1.59843+0.0im
  0.578403+1.00182im    -0.22737+0.0im   0.422912+0.0im     -0.179284+0.0im
   0.19807+0.343067im  -0.641775+0.0im    1.18317+0.0im  …     -1.282+0.0im
 -0.194669-0.337176im  0.0859088+0.0im    1.72777+0.0im      -0.52552+0.0im

In [238]:
B = randn(n+1, r);
B2 = khatri_rao(B, 2)

16×8 Matrix{Float64}:
  2.66711    0.421267    1.33101     0.0535715    …   1.11561      0.922417
  0.862402   0.794348   -0.747954   -0.223965        -0.00448823   1.01593
 -0.916368  -0.152358    0.250709   -0.00465867       1.3106       0.727645
 -0.574944  -0.197443    0.683081   -0.435502         0.742269    -0.425718
  0.862402   0.794348   -0.747954   -0.223965        -0.00448823   1.01593
  0.278855   1.49783     0.42031     0.936328     …   1.80567e-5   1.11892
 -0.296304  -0.287289   -0.140885    0.0194764       -0.0052727    0.801412
 -0.185906  -0.372301   -0.383854    1.82069         -0.00298624  -0.468877
 -0.916368  -0.152358    0.250709   -0.00465867       1.3106       0.727645
 -0.296304  -0.287289   -0.140885    0.0194764       -0.0052727    0.801412
  0.314846   0.0551028   0.0472238   0.000405125  …   1.53967      0.574
  0.197539   0.0714085   0.128665    0.037872         0.872007    -0.335826
 -0.574944  -0.197443    0.683081   -0.435502         0.742269    -0.425

In [243]:
B2

16×8 Matrix{Float64}:
  2.66711    0.421267    1.33101     0.0535715    …   1.11561      0.922417
  0.862402   0.794348   -0.747954   -0.223965        -0.00448823   1.01593
 -0.916368  -0.152358    0.250709   -0.00465867       1.3106       0.727645
 -0.574944  -0.197443    0.683081   -0.435502         0.742269    -0.425718
  0.862402   0.794348   -0.747954   -0.223965        -0.00448823   1.01593
  0.278855   1.49783     0.42031     0.936328     …   1.80567e-5   1.11892
 -0.296304  -0.287289   -0.140885    0.0194764       -0.0052727    0.801412
 -0.185906  -0.372301   -0.383854    1.82069         -0.00298624  -0.468877
 -0.916368  -0.152358    0.250709   -0.00465867       1.3106       0.727645
 -0.296304  -0.287289   -0.140885    0.0194764       -0.0052727    0.801412
  0.314846   0.0551028   0.0472238   0.000405125  …   1.53967      0.574
  0.197539   0.0714085   0.128665    0.037872         0.872007    -0.335826
 -0.574944  -0.197443    0.683081   -0.435502         0.742269    -0.425

In [246]:
IM

16×16 Matrix{Float64}:
 -1.33609    0.929436   0.733894   …   0.0        0.0        -0.0
 -1.67814   -0.245635  -0.787027      -0.0       -0.0         0.0
 -0.136182  -0.303429  -2.11711       -0.0       -0.0         0.0
 -0.991588  -1.31221    0.0251195     -0.0        0.0        -0.0
 -0.0        0.0        0.0            0.0        0.0        -0.0
 -0.0       -0.0       -0.0        …  -0.0       -0.0         0.0
 -0.0       -0.0       -0.0           -0.0       -0.0         0.0
 -0.0       -0.0        0.0           -0.0        0.0        -0.0
 -0.0        0.0        0.0            0.0        0.0        -0.0
 -0.0       -0.0       -0.0           -0.0       -0.0         0.0
 -0.0       -0.0       -0.0        …  -0.0       -0.0         0.0
 -0.0       -0.0        0.0           -0.0        0.0        -0.0
 -0.0        0.0        0.0            0.929436   0.733894   -0.685808
 -0.0       -0.0       -0.0           -0.245635  -0.787027    0.146802
 -0.0       -0.0       -0.0           -0.30

In [251]:
(kron(IM, M)*khatri_rao(B, 3))[1:16, :] + (kron(IM, M)*khatri_rao(B, 3))[17:32, :]

16×8 Matrix{Float64}:
  7.48944    0.176128   2.88817   …   -0.465308  -1.02371   0.632787
  9.97917   -0.717778   2.24827        2.06194   -2.74468  -2.86241
 -1.10729   -0.241688  -0.569063       2.82966   -1.776    -3.0774
  8.05917   -1.13498    1.00452        0.371985  -1.71835  -2.1634
  9.97917   -0.717778   2.24827        2.06194   -2.74468  -2.86241
 13.2966     2.92517    1.75015   …   -9.13716   -7.35882  12.9481
 -1.47538    0.984954  -0.442982     -12.5392    -4.76167  13.9206
 10.7383     4.6254     0.781957      -1.64839   -4.60711   9.7861
 -1.10729   -0.241688  -0.569063       2.82966   -1.776    -3.0774
 -1.47538    0.984954  -0.442982     -12.5392    -4.76167  13.9206
  0.163708   0.331651   0.112124  …  -17.2079    -3.08114  14.9661
 -1.19152    1.55745   -0.197922      -2.26214   -2.98113  10.5211
  8.05917   -1.13498    1.00452        0.371985  -1.71835  -2.1634
 10.7383     4.6254     0.781957      -1.64839   -4.60711   9.7861
 -1.19152    1.55745   -0.197922    

### More testing

In [73]:
A = permutedims(Complex.(L).^(1/4)).*P
M = randn(4, 4)

4×4 Matrix{Float64}:
  0.795153   0.188006  -1.47739   -0.477558
 -0.293978  -0.266215  -0.163437   2.12686
 -0.182942   1.84974   -0.296851  -0.984526
  0.601992  -0.428329  -0.413356  -1.22955

In [74]:
A3 = krDrop(M*A, 3)

20×8 Matrix{ComplexF64}:
  -2.50006+2.50006im     5.33192+0.0im  -0.0127292+0.0im  …   -0.192927+0.0im
   1.94298-1.94298im    -4.57974+0.0im  -0.0751428+0.0im       0.278582+0.0im
  0.733118-0.733118im     5.3502+0.0im     -0.0436+0.0im      0.0765176+0.0im
  -2.60851+2.60851im     3.29662+0.0im    0.059857+0.0im      -0.291687+0.0im
  -1.51004+1.51004im     3.93366-0.0im   -0.443581+0.0im      -0.402265+0.0im
  -0.56976+0.56976im    -4.59544+0.0im   -0.257379+0.0im  …   -0.110489+0.0im
   2.02727-2.02727im    -2.83156+0.0im    0.353347+0.0im       0.421189-0.0im
 -0.214979+0.214979im    5.36855+0.0im   -0.149339+0.0im      -0.030348+0.0im
   0.76492-0.76492im     3.30793+0.0im    0.205022+0.0im       0.115687-0.0im
  -2.72166+2.72166im     2.03824+0.0im   -0.281468+0.0im      -0.441003+0.0im
   1.17356-1.17356im    -3.37874+0.0im    -2.61854+0.0im  …    0.580861+0.0im
  0.442803-0.442803im    3.94715+0.0im    -1.51935+0.0im       0.159544+0.0im
  -1.57554+1.57554im     2.43211+0.0im 

In [79]:
rank(A3[[2, 5, 6, 7, 13, 15, 16], :])

7

In [80]:
N = randn(3, 5)

3×5 Matrix{Float64}:
  0.184735    0.477819  -0.774007   1.52815   -0.41472
 -0.0633081   1.06724   -0.230193  -0.744804  -0.253023
 -0.526962   -0.185757   0.473658  -0.477415   0.853702

In [82]:
tr(N'*N)

6.6767782908419475

In [89]:
sum(norm.(eachrow(N)).^2)

6.6767782908419475